In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/weather-dataset-rattle-package/weatherAUS.csv


In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix, accuracy_score
import sklearn.metrics as metrics

In [3]:
df = pd.read_csv("/kaggle/input/weather-dataset-rattle-package/weatherAUS.csv")
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [4]:
df.shape

(145460, 23)

In [5]:
df.dtypes

Date              object
Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainToday         object
RainTomorrow      object
dtype: object

In [6]:
df = df[df['Location'] == 'Sydney']

In [7]:
df.shape

(3344, 23)

In [8]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
30176,2008-02-01,Sydney,19.5,22.4,15.6,6.2,0.0,NaN,NaN,S,...,92.0,84.0,1017.6,1017.4,8.0,8.0,20.7,20.9,Yes,Yes
30177,2008-02-02,Sydney,19.5,25.6,6.0,3.4,2.7,NaN,NaN,W,...,83.0,73.0,1017.9,1016.4,7.0,7.0,22.4,24.8,Yes,Yes
30178,2008-02-03,Sydney,21.6,24.5,6.6,2.4,0.1,NaN,NaN,ESE,...,88.0,86.0,1016.7,1015.6,7.0,8.0,23.5,23.0,Yes,Yes
30179,2008-02-04,Sydney,20.2,22.8,18.8,2.2,0.0,NaN,NaN,NNE,...,83.0,90.0,1014.2,1011.8,8.0,8.0,21.4,20.9,Yes,Yes
30180,2008-02-05,Sydney,19.7,25.7,77.4,NaN,0.0,NaN,NaN,NNE,...,88.0,74.0,1008.3,1004.8,8.0,8.0,22.5,25.5,Yes,Yes


In [9]:
df.drop(columns = 'Location', inplace = True)

In [10]:
df.dtypes

Date              object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainToday         object
RainTomorrow      object
dtype: object

In [11]:
df = df.dropna()

## Data Preprocessing

###  One Hot Encoding
First, we need to perform one hot encoding to convert categorical variables to binary variables.

In [12]:
df_sydney_processed = pd.get_dummies(data=df, columns=['RainToday', 'WindGustDir', 'WindDir9am', 'WindDir3pm'])

Next, we replace the values of the 'RainTomorrow' column changing them from a categorical column to a binary column. We do not use the get_dummies method because we would end up with two columns for 'RainTomorrow' and we do not want, since 'RainTomorrow' is our target.

In [13]:
df_sydney_processed.replace(['No', 'Yes'], [0,1], inplace=True)

## Training Data and Test Data

In [14]:
df_sydney_processed.drop('Date',axis=1,inplace=True)

In [15]:
df_sydney_processed = df_sydney_processed.astype(float)

In [16]:
features = df_sydney_processed.drop(columns='RainTomorrow', axis=1)
Y = df_sydney_processed['RainTomorrow']

## Modeling: Linear Regression

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, Y, test_size=0.2, random_state=10)

In [18]:
LinearReg = LinearRegression()
LinearReg.fit(X_train, y_train)

LinearRegression()

In [19]:
predictions = LinearReg.predict(X_test)

In [20]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Calculate Mean Absolute Error (MAE)
LinearRegression_MAE = mean_absolute_error(y_test, predictions)

# Calculate Mean Squared Error (MSE)
LinearRegression_MSE = mean_squared_error(y_test, predictions)

# Calculate R-squared (R²)
LinearRegression_R2 = r2_score(y_test, predictions)

print('LinearRegression_MAE:', LinearRegression_MAE)
print('LinearRegression_MSE:', LinearRegression_MSE)
print('LinearRegression_R2:', LinearRegression_R2)

LinearRegression_MAE: 0.26297947542351696
LinearRegression_MSE: 0.12638768973534364
LinearRegression_R2: 0.2679830050126947


In [21]:
# Create a DataFrame to store the metrics
linear_metrics_df = pd.DataFrame({
    'Metric': ['MAE', 'MSE', 'R2'],
    'Value': [LinearRegression_MAE, LinearRegression_MSE, LinearRegression_R2]
})

# Display the DataFrame
print(linear_metrics_df)

  Metric     Value
0    MAE  0.262979
1    MSE  0.126388
2     R2  0.267983


## Modeling: KNN

In [22]:
from sklearn.neighbors import KNeighborsClassifier

# Create the KNN model with n_neighbors=4
KNN = KNeighborsClassifier(n_neighbors=4)

# Train the KNN model using the training data
KNN.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=4)

In [23]:
predictions = KNN.predict(X_test)

In [24]:
from sklearn.metrics import accuracy_score, jaccard_score, f1_score

# Calculate Accuracy Score
KNN_Accuracy_Score = accuracy_score(y_test, predictions)

# Calculate Jaccard Index
KNN_JaccardIndex = jaccard_score(y_test, predictions, average='weighted')

# Calculate F1 Score
KNN_F1_Score = f1_score(y_test, predictions, average='weighted')


In [25]:
print('KNN_Accuracy_Score:', KNN_Accuracy_Score)
print('KNN_JaccardIndex:', KNN_JaccardIndex)
print('KNN_F1_Score:', KNN_F1_Score)

KNN_Accuracy_Score: 0.8017751479289941
KNN_JaccardIndex: 0.6705987733293354
KNN_F1_Score: 0.7777939812171566


## Modeling: Decision Tree

In [26]:
from sklearn.tree import DecisionTreeClassifier

# Create the Decision Tree model
Tree = DecisionTreeClassifier()

# Train the Decision Tree model using the training data
Tree.fit(X_train, y_train)

DecisionTreeClassifier()

In [27]:
predictions = Tree.predict(X_test)

In [28]:
from sklearn.metrics import accuracy_score, jaccard_score, f1_score

# Make predictions using the trained Decision Tree model
tree_predictions = Tree.predict(X_test)

# Calculate Accuracy Score
Tree_Accuracy_Score = accuracy_score(y_test, tree_predictions)

# Calculate Jaccard Index
Tree_JaccardIndex = jaccard_score(y_test, tree_predictions, average='weighted')

# Calculate F1 Score
Tree_F1_Score = f1_score(y_test, tree_predictions, average='weighted')

In [29]:
print('Tree_Accuracy_Score:', Tree_Accuracy_Score)
print('Tree_JaccardIndex:', Tree_JaccardIndex)
print('Tree_F1_Score:', Tree_F1_Score)

Tree_Accuracy_Score: 0.7485207100591716
Tree_JaccardIndex: 0.6280458608214513
Tree_F1_Score: 0.7579449832522205


## Modeling: Logistic Regression

In [30]:

X_train_logistic, X_test_logistic, y_train_logistic, y_test_logistic = train_test_split(features, Y, test_size=0.2, random_state=1)


In [31]:
from sklearn.linear_model import LogisticRegression

# Create the Logistic Regression model with solver='liblinear'
LR = LogisticRegression(solver='liblinear')

# Train the Logistic Regression model using the training data
LR.fit(X_train_logistic, y_train_logistic)


LogisticRegression(solver='liblinear')

In [32]:
# Use the trained Logistic Regression model to make predictions on the testing data
predictions = LR.predict(X_test_logistic)

# Use the trained Logistic Regression model to predict class probabilities on the testing data
predict_proba = LR.predict_proba(X_test_logistic)


In [33]:
from sklearn.metrics import accuracy_score, jaccard_score, f1_score, log_loss

# Calculate Accuracy Score
LR_Accuracy_Score = accuracy_score(y_test_logistic, predictions)

# Calculate Jaccard Index
LR_JaccardIndex = jaccard_score(y_test_logistic, predictions, average='weighted')

# Calculate F1 Score
LR_F1_Score = f1_score(y_test_logistic, predictions, average='weighted')

# Calculate Log Loss
LR_Log_Loss = log_loss(y_test_logistic, predict_proba)

# Print the scores
print("LR Accuracy Score:", LR_Accuracy_Score)
print("LR Jaccard Index:", LR_JaccardIndex)
print("LR F1 Score:", LR_F1_Score)
print("LR Log Loss:", LR_Log_Loss)


LR Accuracy Score: 0.8550295857988166
LR Jaccard Index: 0.7517142977044777
LR F1 Score: 0.8516254080692177
LR Log Loss: 0.3414532904563015


In [34]:
from sklearn.svm import SVC

# Create the SVM model
SVM = SVC()

# Train the SVM model using the training data
SVM.fit(X_train_logistic, y_train_logistic)


SVC()

In [35]:
# Use the trained SVM model to make predictions on the testing data
predictions = SVM.predict(X_test_logistic)


In [36]:
from sklearn.metrics import accuracy_score, jaccard_score, f1_score

# Calculate Accuracy Score
SVM_Accuracy_Score = accuracy_score(y_test_logistic, predictions)

# Calculate Jaccard Index
SVM_JaccardIndex = jaccard_score(y_test_logistic, predictions, average='weighted')

# Calculate F1 Score
SVM_F1_Score = f1_score(y_test_logistic, predictions, average='weighted')

# Print the scores
print("SVM Accuracy Score:", SVM_Accuracy_Score)
print("SVM Jaccard Index:", SVM_JaccardIndex)
print("SVM F1 Score:", SVM_F1_Score)



SVM Accuracy Score: 0.7455621301775148
SVM Jaccard Index: 0.5558628899548335
SVM F1 Score: 0.6368869722194364


In [37]:
import pandas as pd

# Create a DataFrame to store the metrics for all models
metrics_df = pd.DataFrame({
    'Model': ['Linear Regression', 'Decision Tree', 'Logistic Regression', 'SVM'],
    'Accuracy Score': [LR_Accuracy_Score, Tree_Accuracy_Score, LR_Accuracy_Score, SVM_Accuracy_Score],
    'Jaccard Index': [0, 0, LR_JaccardIndex, SVM_JaccardIndex],  # Note: Decision Tree does not have Jaccard Index
    'F1 Score': [0, Tree_F1_Score, LR_F1_Score, SVM_F1_Score],
    'Log Loss': [0, 0, LR_Log_Loss, 0]  # Note: Decision Tree and SVM do not have Log Loss
})

# Print the DataFrame
print(metrics_df)


                 Model  Accuracy Score  Jaccard Index  F1 Score  Log Loss
0    Linear Regression        0.855030       0.000000  0.000000  0.000000
1        Decision Tree        0.748521       0.000000  0.757945  0.000000
2  Logistic Regression        0.855030       0.751714  0.851625  0.341453
3                  SVM        0.745562       0.555863  0.636887  0.000000
